# 安装
```shell
pip install ragas
```

# 1. AspectCritic

AspectCritic是一个指标类。通过AspectCritic，设置提示词，可以灵活地让LLM做出任何评判。

参数：
- name: str, 名称
- definition: str, 设置提示词！
- llm: BaseRagasLLM, 大语言模型实例



```python
from ragas.metrics import AspectCritic
```


# 2. 样本
单论样本， SingleTurnSample

- user_input: str, 用户输入
- retrieved_contexts: List[str], 检索到的上下文
- reference_contexts: List[str], 参考上下文
- response: str, 响应/回答
- reference: str, 参考答案
- multi_responses: List[str], 多个响应

In [1]:
data = {"id": "968d0465-4d3f-4b5c-8326-58cecc4aa70c", "question": "根据背景信息，2025 年科技行业展望中 AI 浪潮重新定义了全球科技的未来，请问在 2024 年 12 月 13 日这份报告中，有哪几位作者？"}

In [2]:
# 1. 根据id获取到对应的文档片段
import chromadb
client = chromadb.HttpClient(host="localhost", port=8000)
collection = client.get_or_create_collection(name="rag-video-collection")
data_info = collection.get(ids=["968d0465-4d3f-4b5c-8326-58cecc4aa70c"])
data_info

{'ids': ['968d0465-4d3f-4b5c-8326-58cecc4aa70c'],
 'embeddings': None,
 'metadatas': [{'source': '科技行业 2025 年展望.txt',
   '作者': '沈岱,马智焱,黄佳琦',
   '发表时间': '2024 年 12 月 13 日',
   '文章标题': '科技行业2025年展望'}],
 'documents': ['科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间'],
 'data': None,
 'uris': None,
 'included': ['documents', 'metadatas']}

In [3]:
data = {
    "id": "968d0465-4d3f-4b5c-8326-58cecc4aa70c", 
    "page_content": "科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间",
    "question": "根据背景信息，2025 年科技行业展望中 AI 浪潮重新定义了全球科技的未来，请问在 2024 年 12 月 13 日这份报告中，有哪几位作者？"
}

In [4]:
from ragas import SingleTurnSample

sample = SingleTurnSample(
    user_input=data.get("question", ""),
    reference_contexts=[data.get("page_content", "")],
)


/opt/miniconda3/envs/rag-video/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sample

SingleTurnSample(user_input='根据背景信息，2025 年科技行业展望中 AI 浪潮重新定义了全球科技的未来，请问在 2024 年 12 月 13 日这份报告中，有哪几位作者？', retrieved_contexts=None, reference_contexts=['科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间'], response=None, multi_responses=None, reference=None, rubrics=None)

In [6]:
from ragas.metrics import AspectCritic
from langchain_ollama import ChatOllama
from ragas.llms import LangchainLLMWrapper

llm = ChatOllama(model="qwen2.5:latest")

metric = AspectCritic(
    name="relevance_assessment",
    definition="判断问题和上下文是否相关？",
    llm=LangchainLLMWrapper(llm),
)

await metric.single_turn_ascore(sample)

1

In [ ]:
Evaluate the Input based on the criterial defined. Use only 'Yes' (1) and 'No' (0) as verdict.
Criteria Definition: 判断问题和上下文是否相关？
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"reason": {"description": "Reason for the verdict", "title": "Reason", "type": "string"}, "verdict": {"description": "The verdict (0 or 1) for the submission", "title": "Verdict", "type": "integer"}}, "required": ["reason", "verdict"], "title": "AspectCriticOutput", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

-----------------------------

Now perform the same with the following input
input: {
    "user_input": "根据背景信息，2025 年科技行业展望中 AI 浪潮重新定义了全球科技的未来，请问在 2024 年 12 月 13 日这份报告中，有哪几位作者？",
    "reference_contexts": [
        "科技行业 2025 年展望:AI 浪潮重新 定义全球科技的未来\n\n发表时间：2024 年 12 月 13 日\n作者：\n(1) 沈岱 - 首席科技分析师 - sia_huang@spdbi.com - (852) 2809 0355\n(2) 马智焱 - 科技分析师 - ivy_ma@spdbi.com - (852) 2809 0300\n(3) 黄佳琦 - 科技分析师 - sia_huang@spdbi.com  - (852) 2809 0355\n\n一、全球 AI 行业具备较大成长空间"
    ]
}
Output: 